<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [2]:
!pip install psycopg2

In [3]:
import lxml
import pandas as pd
import psycopg2

lxml.__version__

'6.0.2'

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python

DBNAME = "project_sql"
USER = "skillfactory"
PASSWORD = "cCkxxLVrDE8EbvjueeMedPKt"
HOST = "84.201.134.129"
PORT = 5432


In [8]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [17]:
# текст запроса
query_3_1 = '''
SELECT COUNT(*) AS vacancies_cnt
FROM vacancies
'''


In [18]:
# результат запроса
pd.read_sql_query(query_3_1, connection)


,vacancies_cnt
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [19]:
# текст запроса
query_3_2 = '''
SELECT COUNT(*) AS employers_cnt
FROM employers
'''


In [20]:
# результат запроса
pd.read_sql_query(query_3_2, connection)


,employers_cnt
0,23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [21]:
# текст запроса
query_3_3 = '''
SELECT COUNT(*) AS areas_cnt
FROM areas
'''


In [22]:
# результат запроса
pd.read_sql_query(query_3_3, connection)


,areas_cnt
0,1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [23]:
# текст запроса
query_3_4 = '''
SELECT COUNT(*) AS industries_cnt
FROM industries
'''


In [24]:
# результат запроса
pd.read_sql_query(query_3_4, connection)


,industries_cnt
0,294


***

# выводы по предварительному анализу данных


В базе данных представлено **49 197 вакансий** и **23 501 работодатель**, что обеспечивает возможность проведения **статистически достоверного анализа** состояния рынка труда.

Информация охватывает **1 362 региона** и **294 отрасли деятельности**, свидетельствуя о **широком географическом охвате и разнообразии профессиональных сфер**, представленных в наборе данных.



## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [25]:
# текст запроса

query_4_1 = '''
SELECT
    a.name AS area,
    COUNT(v.id) AS cnt
FROM vacancies v
JOIN areas a ON v.area_id = a.id
GROUP BY a.name
ORDER BY cnt DESC
'''


In [26]:
# результат запроса

pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [27]:
# текст запроса

query_4_2 = '''
SELECT COUNT(*) AS vacancies_with_salary
FROM vacancies
WHERE salary_from IS NOT NULL
   OR salary_to IS NOT NULL
'''


In [28]:
# результат запроса

pd.read_sql_query(query_4_2, connection)


,vacancies_with_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [29]:
# текст запроса

query_4_3 = '''
SELECT
    ROUND(AVG(salary_from)) AS avg_salary_from,
    ROUND(AVG(salary_to))   AS avg_salary_to
FROM vacancies
'''


In [30]:
# результат запроса

pd.read_sql_query(query_4_3, connection)


,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [31]:
# текст запроса

query_4_4 = '''
SELECT
    schedule,
    employment,
    COUNT(*) AS cnt
FROM vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC
'''


In [32]:
# результат запроса

pd.read_sql_query(query_4_4, connection)


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [33]:
# текст запроса

query_4_5 = '''
SELECT
    experience,
    COUNT(*) AS cnt
FROM vacancies
GROUP BY experience
ORDER BY cnt ASC
'''


In [34]:
# результат запроса

pd.read_sql_query(query_4_5, connection)


,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

### Выводы по детальному анализу вакансий

Основная доля вакансий сосредоточена в крупнейших мегаполисах, прежде всего в Москве и Санкт-Петербурге, что наглядно отражает **централизацию рынка труда**.

Лишь примерно **половина объявлений** содержит сведения о размере заработной платы, что снижает уровень **прозрачности условий** для соискателей.

Преобладающим форматом занятости остаётся **полный рабочий день**, при этом **удалённый формат** также представлен существенной частью вакансий.

Наиболее востребованными являются специалисты с опытом работы **от одного до трёх лет**, что свидетельствует о **высоком спросе на специалистов уровня middle**.



## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [35]:
# текст запроса

query_5_1 = '''
SELECT
    e.name,
    COUNT(v.id) AS vacancies_cnt
FROM employers e
JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.name
ORDER BY vacancies_cnt DESC
'''


In [36]:
# результат запроса

pd.read_sql_query(query_5_1, connection)


,name,vacancies_cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14761,им. Тьюринга,1
14762,LLC Novalab Tech,1
14763,Компания Мир упаковки,1
14764,Лифинцев Олег Иванович,1


In [37]:
pd.read_sql_query(query_5_1, connection).iloc[[0, 4]]


,name,vacancies_cnt
0,Яндекс,1933
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [38]:
# текст запроса

query_5_2 = '''
SELECT
    a.name AS area,
    COUNT(DISTINCT e.id) AS employers_cnt,
    COUNT(v.id) AS vacancies_cnt
FROM areas a
LEFT JOIN employers e ON a.id = e.area
LEFT JOIN vacancies v ON a.id = v.area_id
WHERE v.id IS NULL
GROUP BY a.name
ORDER BY employers_cnt DESC
'''

In [39]:
# результат запроса

pd.read_sql_query(query_5_2, connection)


,area,employers_cnt,vacancies_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


In [40]:
pd.read_sql_query(query_5_2, connection).head(1)


,area,employers_cnt,vacancies_cnt
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [41]:
# текст запроса

query_5_3 = '''
SELECT
    e.name,
    COUNT(DISTINCT v.area_id) AS areas_cnt
FROM employers e
JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.name
ORDER BY areas_cnt DESC
'''


In [42]:
# результат запроса

pd.read_sql_query(query_5_3, connection)


,name,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [43]:
# текст запроса

query_5_4 = '''
SELECT COUNT(*) AS employers_without_industry
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''


In [44]:
# результат запроса

pd.read_sql_query(query_5_4, connection)


,employers_without_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [45]:
# текст запроса

query_5_5 = '''
SELECT e.name
FROM employers e
JOIN employers_industries ei ON e.id = ei.employer_id
GROUP BY e.name
HAVING COUNT(ei.industry_id) = 4
ORDER BY e.name
OFFSET 2 LIMIT 1
'''


In [46]:
# результат запроса

pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [47]:
# текст запроса

query_5_6 = '''
SELECT COUNT(DISTINCT e.id) AS employers_cnt
FROM employers e
JOIN employers_industries ei ON e.id = ei.employer_id
JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''


In [48]:
# результат запроса

pd.read_sql_query(query_5_6, connection)


,employers_cnt
0,3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [49]:
import requests
import pandas as pd

WIKIPEDIA_PAGE = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"

REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
}

def load_cities_from_html():
    response = requests.get(
        WIKIPEDIA_PAGE + "?oldformat=true",
        headers=REQUEST_HEADERS,
        timeout=30
    )
    response.raise_for_status()
    page_html = response.text

    dataframes = pd.read_html(page_html, flavor="lxml")
    for df in dataframes:
        columns = [str(col) for col in df.columns]
        if any("Город" in col for col in columns):
            return df
    raise ValueError("Table with city column not found")

def load_cities_from_api():
    api_url = "https://ru.wikipedia.org/w/api.php"
    query_params = {
        "action": "parse",
        "page": "Города-миллионеры_России",
        "prop": "text",
        "format": "json",
    }

    response = requests.get(
        api_url,
        params=query_params,
        headers=REQUEST_HEADERS,
        timeout=30
    )
    response.raise_for_status()

    json_data = response.json()
    page_html = json_data["parse"]["text"]["*"]

    dataframes = pd.read_html(page_html, flavor="lxml")
    for df in dataframes:
        columns = [str(col) for col in df.columns]
        if any("Город" in col for col in columns):
            return df
    raise ValueError("Table with city column not found via API")

try:
    city_table = load_cities_from_html()
except Exception:
    city_table = load_cities_from_api()

city_column = [col for col in city_table.columns if "Город" in str(col)][0]

city_names = (
    city_table[city_column]
    .astype(str)
    .str.replace(r"\[.*?\]", "", regex=True)
    .str.strip()
)

million_city_list = city_names.tolist()
len(million_city_list), million_city_list[:5]


(16, ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань'])

In [50]:
cities_sql = ", ".join([f"'{c}'" for c in million_city_list])


In [51]:
# текст запроса

query_5_7 = f'''
WITH yandex_vac AS (
    SELECT
        a.name AS city,
        COUNT(v.id) AS vacancies_cnt
    FROM vacancies v
    JOIN employers e ON v.employer_id = e.id
    JOIN areas a ON v.area_id = a.id
    WHERE e.name = 'Яндекс'
      AND a.name IN ({cities_sql})
    GROUP BY a.name
),
total AS (
    SELECT 'Total' AS city, SUM(vacancies_cnt) AS vacancies_cnt
    FROM yandex_vac
)
SELECT city, vacancies_cnt
FROM yandex_vac

UNION ALL

SELECT city, vacancies_cnt
FROM total

ORDER BY vacancies_cnt;
'''


In [52]:
# результат запроса

pd.read_sql_query(query_5_7, connection)



,city,vacancies_cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


In [53]:
df_5_7 = pd.read_sql_query(query_5_7, connection)
len(df_5_7), df_5_7[df_5_7["city"]=="Total"]["vacancies_cnt"].iloc[0]


(17, np.float64(485.0))

***

# Выводы по анализу работодателей

Проведённое исследование работодателей показало, что рынок вакансий отличается **высокой степенью концентрации**: значительная доля предложений сосредоточена у ограниченного круга крупных компаний. Несколько работодателей формируют существенную часть всех вакансий, что свидетельствует об их **ведущей позиции на рынке труда** и более интенсивной стратегии подбора персонала по сравнению с остальными участниками.

Распределение вакансий по территории страны носит **неравномерный характер**. Большинство предложений сосредоточено в крупнейших экономических центрах — в первую очередь в Москве и Санкт-Петербурге, а также в других городах-миллионниках. Это отражает общую тенденцию концентрации бизнеса, технологических отраслей и рабочих мест в крупных городских агломерациях.

Региональный анализ выявил, что в ряде субъектов присутствуют зарегистрированные работодатели, однако вакансии в этих регионах отсутствуют. Подобная ситуация может быть обусловлена **временным отсутствием найма** либо практикой публикации вакансий в централизованных регионах, не совпадающих с фактическим местом работы.

Большинство компаний размещают вакансии лишь в **небольшом числе регионов**, что типично для малого и среднего бизнеса. В отличие от них, крупные организации характеризуются **широкой географией подбора персонала** и представлены сразу в нескольких регионах, что подтверждает их масштаб и развитую организационную структуру.

Также было установлено, что у значительной части работодателей **не указана сфера деятельности**, что снижает качество исходных данных и может осложнять последующий аналитический и отраслевой анализ рынка труда.

Отдельное рассмотрение вакансий компании «Яндекс» показало, что основная часть предложений сосредоточена в городах-миллионниках. Это указывает на ориентацию компании на крупные экономические и технологические центры и соответствует общей региональной структуре распределения вакансий.


## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [54]:
# текст запроса

query_6_1 = f'''
SELECT COUNT(*) AS cnt
FROM vacancies
WHERE LOWER(name) LIKE '%data%'
   OR LOWER(name) LIKE '%данн%';
'''



In [55]:
# результат запроса

pd.read_sql_query(query_6_1, connection)


,cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [56]:
# текст запроса

query_6_2 = f'''
SELECT COUNT(*) AS cnt
FROM vacancies
WHERE (
        LOWER(name) LIKE '%data scientist%'
     OR LOWER(name) LIKE '%data science%'
     OR LOWER(name) LIKE '%исследователь данных%'
     OR LOWER(name) LIKE '%machine learning%'
     OR LOWER(name) LIKE '%машинн%обучен%'
     OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND (
        LOWER(name) LIKE '%junior%'
     OR experience = 'Нет опыта'
     OR employment = 'Стажировка'
);
'''



In [57]:
# результат запроса

pd.read_sql_query(query_6_2, connection)


,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [58]:
# текст запроса

query_6_3 = f'''
SELECT COUNT(*) AS cnt
FROM vacancies
WHERE (
        LOWER(name) LIKE '%data scientist%'
     OR LOWER(name) LIKE '%data science%'
     OR LOWER(name) LIKE '%исследователь данных%'
     OR LOWER(name) LIKE '%machine learning%'
     OR LOWER(name) LIKE '%машинн%обучен%'
     OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND (
        LOWER(key_skills) LIKE '%sql%'
     OR LOWER(key_skills) LIKE '%postgres%'
);
'''


In [59]:
# результат запроса

pd.read_sql_query(query_6_3, connection)


,cnt
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [60]:
# текст запроса

query_6_4 = f'''
SELECT COUNT(*) AS cnt
FROM vacancies
WHERE (
        LOWER(name) LIKE '%data scientist%'
     OR LOWER(name) LIKE '%data science%'
     OR LOWER(name) LIKE '%исследователь данных%'
     OR LOWER(name) LIKE '%machine learning%'
     OR LOWER(name) LIKE '%машинн%обучен%'
     OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND LOWER(key_skills) LIKE '%python%';
'''


In [61]:
# результат запроса

pd.read_sql_query(query_6_4, connection)


,cnt
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [62]:
# текст запроса

query_6_5 = f'''
SELECT ROUND(AVG(skill_cnt), 2) AS avg_skills
FROM (
    SELECT
        id,
        CASE
            WHEN key_skills IS NULL OR key_skills = '' THEN 0
            ELSE array_length(string_to_array(key_skills, E'\\n'), 1)
        END AS skill_cnt
    FROM vacancies
    WHERE (
            LOWER(name) LIKE '%data scientist%'
         OR LOWER(name) LIKE '%data science%'
         OR LOWER(name) LIKE '%исследователь данных%'
         OR LOWER(name) LIKE '%machine learning%'
         OR LOWER(name) LIKE '%машинн%обучен%'
         OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
    )
) t;
'''


In [63]:
# результат запроса

pd.read_sql_query(query_6_5, connection)


,avg_skills
0,0.9


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [64]:
# текст запроса

query_6_6 = f'''
SELECT
    experience,
    ROUND(AVG(
        CASE
            WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL
                THEN (salary_from + salary_to) / 2
            ELSE COALESCE(salary_from, salary_to)
        END
    )) AS avg_salary
FROM vacancies
WHERE (
        LOWER(name) LIKE '%data scientist%'
     OR LOWER(name) LIKE '%data science%'
     OR LOWER(name) LIKE '%исследователь данных%'
     OR LOWER(name) LIKE '%machine learning%'
     OR LOWER(name) LIKE '%машинн%обучен%'
     OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
)
AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
ORDER BY experience;
'''


In [65]:
# результат запроса

pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,Более 6 лет,157933.0
1,Нет опыта,74643.0
2,От 1 года до 3 лет,144233.0
3,От 3 до 6 лет,256454.0


***

# Выводы по предметному анализу

В сегменте вакансий, относящихся к Data Science, отчётливо наблюдается **стабильная потребность в специалистах по анализу данных**: в названиях значительной части позиций напрямую фигурируют термины *data* и *data science*. При этом рынок предлагает сравнительно **небольшое количество возможностей для начинающих** — даже с учётом стажировок, junior-позиций и вакансий без требований к опыту основная масса предложений ориентирована на кандидатов, уже обладающих практическими навыками.

Среди профессиональных требований ключевую роль играют **Python и SQL (PostgreSQL)** — именно эти технологии чаще всего упоминаются работодателями. Как правило, в вакансиях указывается **ограниченный набор ключевых навыков**, что свидетельствует о прагматичном подходе и концентрации на инструментах, реально используемых в работе.

Размер заработной платы тесно связан с уровнем опыта. Наиболее низкие значения характерны для специалистов без стажа, тогда как **максимальный средний уровень дохода** наблюдается у дата-сайентистов с опытом работы от **3 до 6 лет**, что подтверждает высокий спрос на специалистов **middle-уровня**.


## Общий вывод по проекту

# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

По результатам анализа базы вакансий можно сделать вывод, что рынок труда в целом характеризуется **высокой территориальной концентрацией**. Основная часть предложений сосредоточена в экономически развитых и густонаселённых регионах, где расположены штаб-квартиры компаний, IT-кластеры и офисы крупных работодателей. Это означает, что наибольшее количество возможностей для трудоустройства доступно в крупных городах и агломерациях, тогда как в небольших регионах выбор вакансий существенно ограничен. При этом такая ситуация отражает не отсутствие спроса, а **более низкую плотность бизнеса и рабочих мест** за пределами крупных центров.

Анализ заработных плат демонстрирует характерную для рынка зависимость от уровня опыта. Минимальные предложения ориентированы на кандидатов без стажа, после чего наблюдается заметный рост зарплат на уровне 1–3 лет. Наиболее высокие средние значения в сегменте Data Science приходятся на специалистов с опытом 3–6 лет. Это указывает на повышенный интерес работодателей к специалистам среднего уровня, которые уже способны решать прикладные задачи в промышленной среде — разрабатывать и внедрять модели, работать с данными и инфраструктурой, сопровождать решения в продакшене, — но при этом ещё не перешли в более узкий сегмент архитекторов или лидов, где вакансий меньше и требования более специализированные. Важно также учитывать, что средние показатели могут искажаться за счёт небольшого числа высоких предложений и использования зарплатных вилок, поэтому для более точного анализа целесообразно рассматривать медианные значения и квантили.

Рынок Data Science в представленной базе выглядит **стабильным с точки зрения спроса на прикладные навыки**. Наиболее часто в требованиях встречаются Python и SQL/PostgreSQL, что подтверждает тесную связь задач дата-сайентистов с извлечением, подготовкой и анализом данных, а не только с построением моделей. При этом сравнительно небольшое среднее количество указанных навыков является типичным: работодатели, как правило, фиксируют лишь ключевые технологии, предполагая наличие у кандидатов базовых компетенций по умолчанию — аналитического мышления, понимания статистики и опыта работы с данными.

В качестве дальнейшего развития исследования целесообразно рассмотреть несколько направлений. В частности, можно проанализировать распределение зарплат с использованием медианы и перцентилей (25/75) в разрезе опыта и крупных городов; отдельно оценить долю вакансий с указанным уровнем оплаты и с осторожностью интерпретировать сегменты, где зарплата раскрывается редко; изучить концентрацию работодателей, определив вклад топ-N компаний в общее число вакансий и степень зависимости рынка от крупных игроков. Дополнительно полезно сравнить вакансии, относящиеся непосредственно к Data Science, с более широким спектром позиций, связанных с работой с данными (аналитики, BI-специалисты, data engineers), что позволит получить более целостное представление о рынке данных в целом.
